In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage, AIMessage, BaseOutputParser
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

OpenAI

In [5]:
llmo = OpenAI(model='gpt-3.5-turbo-instruct' ,temperature=0.7)
llmo.invoke("What would be a good company name for a company that makes colorful socks?")

'\n"Vibrant Sox Co."'

Prompt Template and LLMChain

In [7]:
prompt_template = PromptTemplate(input_variables=["country"], 
                                 template="Tell me the capital of {country}")
prompt_template.format(country="France")

'Tell me the capital of France'

In [8]:
chain = LLMChain(llm=llmo, prompt=prompt_template)
chain.run("Germany")

C:\Users\wwwro\AppData\Local\Temp\ipykernel_10268\2366404548.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llmo, prompt=prompt_template)
C:\Users\wwwro\AppData\Local\Temp\ipykernel_10268\2366404548.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Germany")


'\n\nThe capital of Germany is Berlin.'

Combining Multiple Chains Using simple Sequential Chain

In [9]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of {country}")
capital_chain = LLMChain(llm=llmo, prompt=capital_prompt)
famous_template = PromptTemplate(input_variables=['capital'],
                                 template='suggest palces to visit in {capital}')
famous_chain = LLMChain(llm=llmo, prompt=famous_template)

In [10]:
chain_seq = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain_seq.run("Germany")

" Here are some places to visit in Berlin:\n\n1. Brandenburg Gate - one of the most iconic landmarks of Berlin, this neoclassical gate serves as a symbol of the city's reunification.\n\n2. The Berlin Wall - a must-see for history buffs, the remnants of the Berlin Wall serve as a reminder of the city's divided past.\n\n3. Reichstag Building - the seat of the German parliament, this historic building offers stunning views of the city from its glass dome.\n\n4. Museum Island - located on the Spree River, this complex is home to five world-renowned museums, including the Pergamon Museum and the Neues Museum.\n\n5. Checkpoint Charlie - another significant historical site, this former border crossing between East and West Berlin now serves as a museum.\n\n6. Charlottenburg Palace - the largest palace in Berlin, this baroque-style building is surrounded by beautiful gardens and houses a museum with royal collections.\n\n7. Berlin Cathedral - a stunning example of Baroque architecture, this ca

Sequential Chain

In [11]:
capital_prompt = PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of {country}")
capital_chain = LLMChain(llm=llmo, prompt=capital_prompt, output_key='capital')

famous_template = PromptTemplate(input_variables=['capital'],
                                 template='suggest palces to visit in {capital}')
famous_chain = LLMChain(llm=llmo, prompt=famous_template, output_key='places')

In [12]:
chain_seq = SequentialChain(chains=[capital_chain, famous_chain], 
                            input_variables=['country'],
                            output_variables=['capital', 'places'])
chain_seq({"country":"Germany"})

C:\Users\wwwro\AppData\Local\Temp\ipykernel_10268\929704000.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain_seq({"country":"Germany"})


{'country': 'Germany',
 'capital': '\n\nThe capital of Germany is Berlin.',
 'places': " Here are some places you may want to visit while in Berlin:\n\n1. Brandenburg Gate - This iconic landmark is a must-see in Berlin and symbolizes the unity of the city.\n\n2. Reichstag Building - The seat of the German Parliament, this historic building offers guided tours and a stunning view from its dome.\n\n3. Berlin Wall - A visit to the Berlin Wall is a must for any history buff. You can see the remaining sections of the wall and learn about its significance.\n\n4. Museum Island - This UNESCO World Heritage Site is home to five of Berlin's most famous museums, including the Pergamon Museum and the Neues Museum.\n\n5. Checkpoint Charlie - This former border crossing between East and West Berlin is now a popular tourist spot where you can learn about the Cold War.\n\n6. Tiergarten Park - This large park in the heart of the city is perfect for a leisurely stroll or a picnic. It also houses various

Chatmodels with ChatOpenAI

In [13]:
chatllm = ChatOpenAI(model='gpt-3.5-turbo' ,temperature=0.7)

In [14]:
chatllm([
    SystemMessage(content='You are a langchain expert'),
    HumanMessage(content='Return only langchain retaled')
])

C:\Users\wwwro\AppData\Local\Temp\ipykernel_10268\3001775620.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatllm([


AIMessage(content='Langchain is a decentralized platform that enables users to create, manage, and trade personal data in a secure and transparent manner using blockchain technology. It aims to give individuals control over their own data and provide a secure way to share it with others. Langchain utilizes smart contracts and encryption techniques to ensure the privacy and integrity of the data stored on the platform. Users can also earn rewards by sharing their data with trusted parties through the Langchain ecosystem.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 23, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CIohxIwKBW48vbE3H3ZGQlsnLEHpM', 'service_tier':

Prompt Template + LLM +  Output Parsers

In [15]:
class Commasepratedoutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")
    

In [16]:
template = "You are a helpfule assistant. When user give input generate 10 words in a comma seprated list"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [17]:
chain = chatprompt | chatllm | Commasepratedoutput()
chain.invoke({"text":"programming languages"})

['programming',
 ' languages',
 ' code',
 ' software',
 ' development',
 ' computer',
 ' technology',
 ' Java',
 ' Python',
 ' C++']